# Thông tin sinh viên

Họ tên: Nguyễn Nguyên Khôi

MSSV: 21521009

Bài tập số 3

Ngày thực hiện 29-3-2023

# Yêu cầu 1: Phân lớp ảnh dogs vs cats

In [1]:
import numpy as np
from numpy.linalg import norm
import pandas as pd
import cv2 as cv
import os
import matplotlib.pyplot as plt
from scipy.spatial import distance
from scipy import ndimage
from google.colab import drive
from google.colab.patches import cv2_imshow
from sklearn.neighbors import KNeighborsClassifier

In [2]:
drive.mount("/content/gdrive")

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [3]:
dir_dataset = "gdrive/MyDrive/CS231.N22.KHCL/Baitap/Baitap3/Dataset/dogscats_small"
dir_datatrain = os.path.join(dir_dataset, "train")
dir_datatest = os.path.join(dir_dataset, "test")

In [4]:
def CalHist(pathfilename):
  img = cv.imread(pathfilename,0)
  hist = cv.calcHist([img], [0], None, [256], [0,256])
  size = img.shape[0]*img.shape[1]
  hist = hist / size
  return hist

In [5]:
def ReadData(path_to_files):
  feature = []
  labels = []
  for file_name in os.listdir(path_to_files):
    # Take label of the image
    label = file_name.split(os.path.sep)[-1].split(".")[0]
    # -> Insert into list of labels
    if label == "dog":
      labels.append(0)
    else:
      labels.append(1)
    # Calculate image feature
    feature.append(CalHist(path_to_files + "/" + file_name))
    # -> Insert into list of features
  features = np.array(feature)
  labels = np.array(labels)
  return features, labels

In [6]:
X_train, y_train = ReadData(dir_datatrain)
X_test, y_test = ReadData(dir_datatest)
     
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1]))
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1]))

In [7]:
print("Shape of X_train:", X_train.shape)
print("Shape of y_train:", y_train.shape)
     
print("Shape of X_test", X_test.shape)
print("Shape of y_test",y_test.shape)     

Shape of X_train: (42, 256)
Shape of y_train: (42,)
Shape of X_test (20, 256)
Shape of y_test (20,)


In [8]:
model = KNeighborsClassifier(n_neighbors=3)
model.fit(X_train, y_train)
acc = model.score(X_test, y_test)
print("Model's accuracy on test set:", acc)

0.45

# FROM SCRATCH

In [9]:
from math import sqrt

In [10]:
# calculate distance between two vectors
def euclidean_distance(a: np.ndarray, b: np.ndarray) -> float:
    '''calculate distance between two vectors a and b
    '''
    S = 0
    for i in range(len(a)):
      S += pow((a[i] - b[i]), 2)
    return sqrt(S)

In [11]:
# building the KNN classifier
class KNNClassifier:
  def __init__(self, k_neighbors: int):
    self.k_neighbors = k_neighbors
    self.X_train = None
    self.y_train = None

  def predict(self, X) -> np.ndarray:
    y_pred = np.array([])
    for i in range(len(X)):
      distances = dict()
      for j in range(len(self.X_train)):
        distances[j] = euclidean_distance(X[i], self.X_train[j])
      candidates = sorted(distances.items(), key=lambda x: x[1])[:self.k_neighbors]
      labels = [0, 0]
      for j in candidates:
        if self.y_train[j[0]] == 0: labels[0] += 1
        else: labels[1] += 1
      if labels[0] >= labels[1]:
        y_pred = np.concatenate((y_pred, [0]))
      else:
        y_pred = np.concatenate((y_pred, [1]))
    return y_pred

  def fit(self, X: np.ndarray, y: np.ndarray):
    '''X and y are required to be standardized
    Categorical values in y are needed to be converted to 0 and 1
    ''' 
    try:
      self.X_train = X
      self.y_train = y
    except:
      print("X and y must be numpy arrays")

In [12]:
# function calculating error rate
def error_rate(prediction, actual):
  num_of_points = len(prediction)
  num_of_errors = 0
  for i in range(num_of_points):
    if prediction[i] != actual[i]: num_of_errors += 1
  return num_of_errors/num_of_points

In [13]:
classifier = KNNClassifier(k_neighbors=3)
classifier.fit(X_train, y_train)

In [14]:
y_test_pred = classifier.predict(X_test)

In [16]:
print("Model's accuracy on test set:", 1-error_rate(y_test_pred, y_test))

Model's accuracy on test set: 0.44999999999999996
